In [ ]:
import os
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\Figure\TRASH')

In [ ]:
from collections import Counter
import random

In [ ]:
sample = "Glycine_max"
prefix = "Gm"
s = 'G'
all_repeats = []
# with open(f"all.repeats.from.{sample}.chr.fasta.csv", 'r')as f:
with open(f"all.repeats.from.{sample}.fasta.csv", 'r')as f:
    next(f)
    for line in f:
        fields = line.rstrip().replace('"','').split(',')
        all_repeats.append(fields)
print(len(all_repeats))

In [ ]:
all_repeats_fil_len = [i for i in all_repeats if int(i[2]) >= 50]
print(len(all_repeats_fil_len))

In [ ]:
cnt = Counter(i[2] for i in all_repeats_fil_len)
sorted_items = sorted(cnt.items(), key=lambda x: x[0])
sorted_items

In [ ]:
with open(f"{prefix}.count.txt", 'w')as f:
    f.write(f"Repeat\tLength\tNumber\n")
    for k,v in sorted_items:
        repeat = f"{prefix}{k}"
        f.write(f"{repeat}\t{k}\t{v}\n")

In [ ]:
len_left = 400
len_right = 470
all_repeats_fil_len_selected = [i for i in all_repeats_fil_len if int(i[2]) >= len_left and int(i[2]) <= len_right]
len(all_repeats_fil_len_selected)

In [ ]:
repeat_len = 455
len_left = repeat_len - 2
len_right = repeat_len + 1
all_repeats_fil_len_selected_1 = [i for i in all_repeats_fil_len if int(i[2]) >= len_left and int(i[2]) <= len_right]
with open(f"{prefix}{repeat_len}.fasta", 'w')as f:
    count = 0
    for i in all_repeats_fil_len_selected_1:
        count += 1
        seq_name1 = f">{i[7]}:{i[0]}-{i[1]}_{i[4]}"
        seq_name2 = f">S{str(count).zfill(5)}"
        sequence = i[3]
        f.write(f"{seq_name2}\n")
        f.write(f"{sequence}\n")

In [ ]:
from collections import defaultdict

In [ ]:
groups = defaultdict(list)
for rec in all_repeats_fil_len:
    length = int(rec[2])
    if len_left <= length <= len_right:
        chr_name = rec[7]
        groups[chr_name].append(rec)

In [ ]:
len(groups)

In [ ]:
repeat_len = 335
len_left = repeat_len - 1
len_right = repeat_len + 1

groups = defaultdict(list)
for rec in all_repeats_fil_len:
    length = int(rec[2])
    if len_left <= length <= len_right:
        chr_name = rec[7]
        groups[chr_name].append(rec)

with open(f"{prefix}{repeat_len}_id_mapping.txt", 'w') as id_mapping_file:
    id_mapping_file.write("Original_ID\tModified_ID\n")

    for chr_name, records in groups.items():
        records.sort(key=lambda rec: int(rec[0]))  

        out_fname = f"{prefix}{repeat_len}_{chr_name}.fasta"
        with open(out_fname, 'w') as f:
            for idx, rec in enumerate(records, start=1):
                seq_name1 = f">{rec[7]}:{rec[0]}-{rec[1]}_{rec[4]}"
                seq_name2 = f">{s}4{chr_name}_S{idx:05d}"
                sequence = rec[3]

                f.write(seq_name2 + "\n")
                f.write(sequence + "\n")
                id_mapping_file.write(f"{seq_name1[1:]}\t{seq_name2[1:]}\n")

        with open(f"{prefix}{repeat_len}_chr_count.txt", 'a')as f:
            f.write(f"{chr_name}\t{len(records)}\n")

        print(f"Wrote {len(records)} records to {out_fname}")

print(f"Original and modified IDs have been written to {prefix}{repeat_len}_id_mapping.txt")

In [ ]:
def FastaToDict(file):
    fasta = {}
    with open(file, 'r')as f:
        lines = f.readlines()
    for line in lines:
        if line[0] == '>':
            key = line.strip()
            fasta[key] = []
        else:
            fasta[key].append(line.strip())
    for key,value in list(fasta.items()):
        fasta[key] = ''.join(value)
    return fasta

In [ ]:
def merge_fasta_files(input_folder, output_file):
    fasta_files = [f for f in os.listdir(input_folder) if f.startswith(f"{prefix}455_") and f.endswith(".fasta")]

    with open(output_file, 'w') as outfile:
        for fasta_file in fasta_files:
            input_file_path = os.path.join(input_folder, fasta_file)
            with open(input_file_path, 'r') as infile:
                outfile.write(infile.read())
            print(f"Added {fasta_file} to {output_file}")

    print(f"All files have been merged into {output_file}")

In [ ]:
merge_fasta_files("./",  f"{prefix}{repeat_len}_merge.fasta")

In [ ]:
seq = FastaToDict(f"{prefix}{repeat_len}_merge.fasta")
with open(f"{prefix}{repeat_len}_mostcopy.fasta", 'w')as f:
    for i in Counter(seq.values()).most_common(1):
        f.write(f">{prefix}{repeat_len}_mostcopy_{i[1]}\n")
        f.write(f"{i[0]}\n")

In [ ]:
def select_random_items(input_dict, num_items, output_file):
    if num_items > len(input_dict):
        print("Error: The number of items to select exceeds the number of items in the dictionary.")
        return

    selected_items = random.sample(list(input_dict.items()), num_items)
    selected_dict = dict(selected_items)
    
    with open(output_file, 'w') as f:
        for k,v in selected_dict.items():
            f.write(f"{k}\n")
            f.write(f"{v}\n")
        
    print(f"Random {num_items} key-value pairs have been written to {output_file}.")

In [ ]:
# random_list = [('Chr01',280),('Chr04',27),('Chr05',421),('Chr06',180),('Chr07',432),('Chr08',233),('Chr10',118)]
# random_list = [('Chr01',285),('Chr04',29),('Chr05',376),('Chr06',169),('Chr07',442),('Chr08',197),('Chr10',115)]
# random_list = [('Chr01',140),('Chr04',14),('Chr05',211),('Chr06',90),('Chr07',216),('Chr08',116),('Chr10',59)]
# random_list = [('Chr01',207),('Chr04',290),('Chr05',29),('Chr06',25),('Chr07',63),('Chr08',62),('Chr10',157)]
# random_list = [('Chr02',76),('Chr04',338),('Chr06',145),('Chr11',80)]
# random_list = [('Chr06',142),('Chr11',158)]
# random_list = [('chr03',34),('chr04',35)]
# random_list = [('Chr01',36),('Chr02',11),('Chr03',20),('Chr04',130),('Chr05',30),('Chr07',13),('Chr08',20),('Chr10',71),('Chr11',70)]
random_list = [('Chr19',40),('Chr09',10)]
for chrom,num in random_list:
    fasta_file = f"{prefix}{repeat_len}_{chrom}.fasta"
    seq = FastaToDict(fasta_file)
    out_file = f"{prefix}{repeat_len}_{chrom}_random_{num}.fasta"
    select_random_items(seq, num, out_file)

In [ ]:
#mostcopy
sample = "Glycine_max"
prefix = "Gm"
all_repeats = []
with open(f"all.repeats.from.{sample}.fasta.csv", 'r')as f:
    next(f)
    for line in f:
        fields = line.rstrip().replace('"','').split(',')
        all_repeats.append(fields)
print(len(all_repeats))
all_repeats_fil_len = [i for i in all_repeats if int(i[2]) >= 50]
print(len(all_repeats_fil_len))
cnt = Counter(i[2] for i in all_repeats_fil_len)
sorted_items = sorted(cnt.items(), key=lambda x: x[0])
with open(f"{prefix}.count.txt", 'w')as f:
    f.write(f"Repeat\tLength\tNumber\n")
    for k,v in sorted_items:
        repeat = f"{prefix}{k}"
        f.write(f"{repeat}\t{k}\t{v}\n")

In [ ]:
sorted_items

In [ ]:
repeat_len = 92
len_left = repeat_len - 1
len_right = repeat_len + 1
all_repeats_fil_len_selected_1 = [i for i in all_repeats_fil_len if int(i[2]) >= len_left and int(i[2]) <= len_right]
with open(f"{prefix}{repeat_len}_mostcopy.fasta", 'w')as f:
    for i in Counter(j[3] for j in all_repeats_fil_len_selected_1).most_common(1):
        f.write(f">{prefix}{repeat_len}_mostcopy_{i[1]}\n")
        f.write(f"{i[0]}\n")

In [ ]:
#ALLmostcopy.fasta
sample = "Vigna_radiata"
prefix = "Vr"
all_repeats = []
with open(f"all.repeats.from.{sample}.fasta.csv", 'r')as f:
    next(f)
    for line in f:
        fields = line.rstrip().replace('"','').split(',')
        all_repeats.append(fields)
print(len(all_repeats))
all_repeats_fil_len = [i for i in all_repeats if int(i[2]) >= 50]
print(len(all_repeats_fil_len))
cnt = Counter(i[2] for i in all_repeats_fil_len)
sorted_items = sorted(cnt.items(), key=lambda x: x[0])
for i in sorted_items:
    repeat_len = int(i[0])
    all_repeats_fil_len_selected_1 = [i for i in all_repeats_fil_len if int(i[2]) == repeat_len]
    with open(f"{prefix}_ALLmostcopy.fasta", 'a')as f:
        for i in Counter(j[3] for j in all_repeats_fil_len_selected_1).most_common(1):
            f.write(f">{prefix}{repeat_len}_mostcopy_{i[1]}\n")
            f.write(f"{i[0]}\n")

In [ ]:
#random hor
sample = "HJD.chr"
prefix = "HJD"
s = 'H'
all_repeats = []
with open(f"all.repeats.from.{sample}.fasta.csv", 'r')as f:
    next(f)
    for line in f:
        fields = line.rstrip().replace('"','').split(',')
        all_repeats.append(fields)
print(len(all_repeats))
all_repeats_fil_len = [i for i in all_repeats if int(i[2]) >= 50]
print(len(all_repeats_fil_len))
cnt = Counter(i[2] for i in all_repeats_fil_len)
sorted_items = sorted(cnt.items(), key=lambda x: x[0])

repeat_len = 315
len_left = repeat_len - 1
len_right = repeat_len + 1

groups = defaultdict(list)
for rec in all_repeats_fil_len:
    length = int(rec[2])
    if len_left <= length <= len_right:
        chr_name = rec[7]
        groups[chr_name].append(rec)

with open(f"{prefix}{repeat_len}_id_mapping.txt", 'w') as id_mapping_file:
    id_mapping_file.write("Original_ID\tModified_ID\n")

    for chr_name, records in groups.items():
        records.sort(key=lambda rec: int(rec[0]))  

        out_fname = f"{prefix}{repeat_len}_{chr_name}.fasta"
        with open(out_fname, 'w') as f:
            for idx, rec in enumerate(records, start=1):
                seq_name1 = f">{rec[7]}:{rec[0]}-{rec[1]}_{rec[4]}"
                seq_name2 = f">{s}{chr_name}_S{idx:05d}"
                sequence = rec[3]

                f.write(seq_name2 + "\n")
                f.write(sequence + "\n")
                id_mapping_file.write(f"{seq_name1[1:]}\t{seq_name2[1:]}\n")

        with open(f"{prefix}{repeat_len}_chr_count.txt", 'a')as f:
            f.write(f"{chr_name}\t{len(records)}\n")

        print(f"Wrote {len(records)} records to {out_fname}")

print(f"Original and modified IDs have been written to {prefix}{repeat_len}_id_mapping.txt")

In [ ]:
#HJD338
random_list = [('Chr02',76),('Chr04',338),('Chr06',144),('Chr11',80)]
for chrom,num in random_list:
    fasta_file = f"{prefix}{repeat_len}_{chrom}.fasta"
    seq = FastaToDict(fasta_file)
    out_file = f"{prefix}{repeat_len}_{chrom}_random_{num}.fasta"
    select_random_items(seq, num, out_file)

In [ ]:
#Gm335
random_list = [('Chr02',76),('Chr04',338),('Chr06',144),('Chr11',80)]
for chrom,num in random_list:
    fasta_file = f"{prefix}{repeat_len}_{chrom}.fasta"
    seq = FastaToDict(fasta_file)
    out_file = f"{prefix}{repeat_len}_{chrom}_random_{num}.fasta"
    select_random_items(seq, num, out_file)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

## process afa

In [ ]:
sequences = FastaToDict("Gm335.all.afa")

In [ ]:
ref_id = ">Gm335_Chr19_mostcopy_156"
ref_seq = sequences[ref_id]
all_seqs = sequences.items()
num_seqs = len(all_seqs)
seq_len = len(ref_seq)

assert all(len(seq) == seq_len for seq in sequences.values()), "序列长度不一致"

# 第一步：找出符合条件的保留位置（gap比例 ≥ 10%）
valid_positions = []
for i in range(seq_len):
    if ref_seq[i] in "ACGT":continue
    column = [seq[i] for _, seq in all_seqs]
    gap_ratio = column.count('-') / num_seqs
    if gap_ratio >= 0.1 :
        valid_positions.append(i)

In [ ]:
invalid_positions = [i for i in range(seq_len) if i not in valid_positions]
len(invalid_positions)

In [ ]:
# 第二步：统计每个序列在这些位置与参考序列的 mismatch 数量
results = []
for seq_id, seq in all_seqs:
    mismatch_count = 0
    for pos in invalid_positions:
        base = seq[pos]
        ref_base = ref_seq[pos]
        # if base != '-' and base != ref_base:
        if base != ref_base:
            mismatch_count += 1
    results.append([seq_id, mismatch_count])

In [ ]:
out_file = "Gm335_Chr19_mismatch.txt"
with open(out_file, 'w')as f:
    for i in results:
        f.write(f"{i[0]}\t{i[1]}\n")

In [ ]:
len(ref_seq)

In [ ]:
len(valid_positions)

In [ ]:
150/454

In [ ]:
#get the most copy number sequence
# fa_file = "HJD338_Chr02.fasta"
# fa_file = "HJD338_Chr04.fasta"
# fa_file = "HJD338_Chr06.fasta"
# fa_file = "HJD338_Chr11.fasta"
# fa_file = "Gm335_Chr09.fasta"
fa_file = "HJD455_Chr10.fasta"
out_prefix = fa_file.replace('.fasta','')
seq = FastaToDict(fa_file )
with open(f"{out_prefix}_MCoNum.fasta", 'w')as f:
    for i in Counter(seq.values()).most_common(1):
        f.write(f">{out_prefix}_mostcopy_{i[1]}\n")
        f.write(f"{i[0]}\n")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\Figure\TRASH\HJD-FC6-455')

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
df = pd.read_excel("HJD455_plot.xlsx", sheet_name="Sheet2")

# === 预处理 ===
data_dict = {}

for col in df.columns:
    # 去除空值
    values = df[col].dropna()
    # 归一化处理：1 - x/455
    values = (1 - values / 455) * 100
    # 删除离群值（IQR方法）
    Q1 = values.quantile(0.25)
    Q3 = values.quantile(0.75)
    IQR = Q3 - Q1
    filtered_values = values[(values >= Q1 - 1.5 * IQR) & (values <= Q3 + 1.5 * IQR)]
    data_dict[col] = filtered_values

# === 整理为长格式DataFrame用于绘图 ===
long_df = pd.concat([
    pd.DataFrame({ "Group": [key]*len(val), "Value": val.values })
    for key, val in data_dict.items()
], ignore_index=True)

In [ ]:
custom_colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99"]
column_names = list(data_dict.keys())

# 创建颜色映射
color_map = dict(zip(column_names, custom_colors))

# === 绘图 ===
plt.figure(figsize=(5, 5.5))
ax = sns.boxplot(data=long_df, x="Group", y="Value", palette=color_map)

# 不显示 x 轴标签
ax.set_xticklabels([])

# 创建 legend 手动映射
from matplotlib.patches import Patch
legend_handles = [Patch(color=color_map[name], label=name) for name in column_names]
plt.legend(
    handles=legend_handles,
    loc='lower center',
    bbox_to_anchor=(0.5, -0.15), #和主体间距
    ncol=2,
    frameon=False,
    fontsize=9,
    title_fontsize=10,
    handlelength=1.2,
    handleheight=0.6,
    handletextpad=0.5,
    columnspacing=0.8
)

# 样式调整
plt.grid(False)
sns.despine(top=False, right=False, left=False, bottom=False)
# plt.title("Boxplot (Normalized, Outliers Removed)", fontsize=14)
plt.xlabel("")
plt.ylabel("Similarity (%)", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout()
name = "HF455"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
df = pd.read_excel("HJD338_plot.xlsx", sheet_name="Sheet1")

# === 预处理 ===
data_dict = {}

for col in df.columns:
    # 去除空值
    values = df[col].dropna()
    # 归一化处理：1 - x/455
    values = values * 100
    # 删除离群值（IQR方法）
    Q1 = values.quantile(0.25)
    Q3 = values.quantile(0.75)
    IQR = Q3 - Q1
    filtered_values = values[(values >= Q1 - 1.5 * IQR) & (values <= Q3 + 1.5 * IQR)]
    data_dict[col] = filtered_values

# === 整理为长格式DataFrame用于绘图 ===
long_df = pd.concat([
    pd.DataFrame({ "Group": [key]*len(val), "Value": val.values })
    for key, val in data_dict.items()
], ignore_index=True)
custom_colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99"]
column_names = list(data_dict.keys())

# 创建颜色映射
color_map = dict(zip(column_names, custom_colors))

# === 绘图 ===
plt.figure(figsize=(5, 5.5))
ax = sns.boxplot(data=long_df, x="Group", y="Value", palette=color_map)

# 不显示 x 轴标签
ax.set_xticklabels([])

# 创建 legend 手动映射
from matplotlib.patches import Patch
legend_handles = [Patch(color=color_map[name], label=name) for name in column_names]
plt.legend(
    handles=legend_handles,
    loc='lower center',
    bbox_to_anchor=(0.5, -0.15), #和主体间距
    ncol=2,
    frameon=False,
    fontsize=9,
    title_fontsize=10,
    handlelength=1.2,
    handleheight=0.6,
    handletextpad=0.5,
    columnspacing=0.8
)

# 样式调整
plt.grid(False)
sns.despine(top=False, right=False, left=False, bottom=False)
# plt.title("Boxplot (Normalized, Outliers Removed)", fontsize=14)
plt.xlabel("")
plt.ylabel("Similarity (%)", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.tight_layout()
name = "HF338"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\Figure\TRASH\fig')

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

df = pd.read_excel("Count.xlsx", sheet_name="FC6")

# bar_color = "#1f78b4"
bar_color = "#33a02c"

plt.figure(figsize=(4, 2))
bars = plt.bar(
    x=df["Length"],
    height=df["Number"],
    color=bar_color,
    width=1,  # 宽度设为1使柱子无间隙
    align="center"
)

# === 样式调整 ===
plt.ylim(bottom=df["Number"].min() * 0.95)  # Y轴起始点略高于0
# plt.xlabel("Length", fontsize=12)
plt.ylabel("Number", fontsize=12)
# plt.title("Repeat Length Distribution", fontsize=14)

# 去掉顶部和右侧边框，仅保留左和下
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_visible(True)

# 不显示grid
plt.grid(False)

# 去掉 x 轴和 y 轴的刻度线（可选）
# ax.tick_params(axis='both', which='both', length=0)

plt.tight_layout()
name = "FC6count"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

df = pd.read_excel("Count.xlsx", sheet_name="FC6V", header=None)
data = df[0]

# hist_color = "#1f78b4"
hist_color = "#33a02c"  # 可修改为自定义颜色，如 "#4C72B0"

# === 绘图 ===
plt.figure(figsize=(4, 2))
counts, bins, patches = plt.hist(
    data,
    bins='auto',           # 自动分箱，可自定义
    color=hist_color,
    edgecolor='black',     # 让柱子边缘可见（可选）
    linewidth=0.5,
)

# 设置柱子无间距（强制每个柱子挨着）
for patch in patches:
    patch.set_linewidth(0)
    patch.set_edgecolor(hist_color)

# === 坐标轴调整 ===
plt.ylim(bottom=min(counts) * 0.95)  # Y轴略高于0
# plt.xlabel("Value", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
# plt.title("Histogram of HJDV Values", fontsize=14)

# 只显示左边和下边框
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_visible(True)

# 不显示grid
plt.grid(False)

plt.tight_layout()
name = "FC6Variant"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

# === 读取数据 ===
df = pd.read_excel("Count.xlsx", sheet_name="HJD455")

# === 准备数据 ===
labels = df['chrom']
hjd_values = df['HJD']
fc6_values = df['FC6']

x = np.arange(len(labels))  # 每组的 x 位置
width = 0.35  # 每个柱子的宽度

# === 自定义颜色 ===
color_hjd = "#1f78b4"  # 蓝色系
color_fc6 = "#33a02c"  # 橙色系

# === 绘图 ===
fig, ax = plt.subplots(figsize=(4, 2))
bars1 = ax.bar(x - width/2, hjd_values, width, label='HJD', color=color_hjd)
bars2 = ax.bar(x + width/2, fc6_values, width, label='FC6', color=color_fc6)

# === 设置坐标轴 & 外观 ===
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel("Number", fontsize=12)
# ax.set_title("Grouped Bar Chart: HJD vs FC6", fontsize=14)

# 只保留左和下边框
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_visible(True)

# 不显示 grid
ax.grid(False)

# 图例
ax.legend()

plt.tight_layout()
name = "HJD455_chrom"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

# === 读取数据 ===
df = pd.read_excel("Count.xlsx", sheet_name="FC6315")

# === 准备数据 ===
labels = df['chrom']
hjd_values = df['HJD']
fc6_values = df['FC6']

x = np.arange(len(labels))  # 每组的 x 位置
width = 0.35  # 每个柱子的宽度

# === 自定义颜色 ===
color_hjd = "#1f78b4"  # 蓝色系
color_fc6 = "#33a02c"  # 橙色系

# === 绘图 ===
fig, ax = plt.subplots(figsize=(4, 2))
bars1 = ax.bar(x - width/2, hjd_values, width, label='HJD', color=color_hjd)
bars2 = ax.bar(x + width/2, fc6_values, width, label='FC6', color=color_fc6)

# === 设置坐标轴 & 外观 ===
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel("log$_{10}$(Number)", fontsize=12)
# ax.set_title("Grouped Bar Chart: HJD vs FC6", fontsize=14)

# 只保留左和下边框
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_visible(True)
# ax.set_ylim(bottom=-100)
# 不显示 grid
ax.grid(False)
ax.legend()

plt.tight_layout()
name = "FC6315_chrom"
plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\Figure\TRASH\fig1')

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

name = "Pv"
df = pd.read_excel("Count.xlsx", sheet_name=name)

# bar_color = "#1f78b4"
# bar_color = "#33a02c"
bar_color = "black"

plt.figure(figsize=(4, 2))
bars = plt.bar(
    x=df["Length"],
    height=df["Number"],
    color=bar_color,
    width=1,  # 宽度设为1使柱子无间隙
    align="center"
)

# === 样式调整 ===
plt.ylim(bottom=df["Number"].min() * 0.95)  # Y轴起始点略高于0
# plt.xlabel("Length", fontsize=12)
plt.ylabel("Number", fontsize=12)
# plt.title("Repeat Length Distribution", fontsize=14)

# 去掉顶部和右侧边框，仅保留左和下
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_visible(True)

# 不显示grid
plt.grid(False)

# 去掉 x 轴和 y 轴的刻度线（可选）
# ax.tick_params(axis='both', which='both', length=0)

plt.tight_layout()

plt.savefig(f'{name}.jpg', dpi=300, bbox_inches='tight')
plt.savefig(f'{name}.svg', bbox_inches='tight')
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.show()